## Морфопарсинг

Установка нужных для него библиотек:

**pymorphy2**

*pip install pymorphy2*

*pip install -U pymorphy2-dicts-ru*  # необязательно: для обновления словаря

*Colab*: устанавливается без проблем. 

**Mystem**

*pip install git+https://github.com/nlpub/pymystem3* 

(возможно, уже перестало работать для новых версий питона: можно установить просто как pip install pymystem3, но тогда лучше не запускать на нем длинные тексты с переносами на новую строку.)

*Colab*: гарантированно не работает. 

**rnnmorph**

*pip install rnnmorph*

Может козлить во время установки. Иногда, если плохо установился и не работает, приходится его удалять командой pip uninstall rnnmorph (обязательно в консоли от имени администратора!). Периодически, если обновляется версия tensorflow, может перестать работать &ndash; но Илья Гусев обычно вскоре обновляет и свой парсер, так что достаточно следить за новостями в [его гитхабе](https://github.com/IlyaGusev/rnnmorph) &ndash; или просто подождать светлых времен.

*Colab*: обычно работает без проблем.

**pyconll**

*pip install pyconll*

*Colab*: устанавливается без проблем. 

Почти все нижеописанные инструменты в питоне устроены довольно однообразно: имеется основной класс "парсер", экземпляр которого вам нужно создать, прежде чем что-то парсить. То есть, условно говоря, из набора машинок берете ту конкретную, которая вас повезет. Когда создан экземпляр класса, ему уже можно скармливать свои тексты. 

Самые простые &ndash; правиловые морфопарсеры. Для русского языка их два: pymorphy2 и pymystem3. Pymorphy был создан Михаилом Коробовым (вот его известная [статья на хабре](https://habr.com/ru/post/176575/)) как аналог Майстем. Он работает на словаре и использует тагсет [OpenCorpora](http://opencorpora.org/)), а также статистику, предпосчитанную на этом корпусе. 
Как устроен pymorphy2?

In [5]:
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

parse = morph.parse('студентки')
parse

[Parse(word='студентки', tag=OpencorporaTag('NOUN,anim,femn sing,gent'), normal_form='студентка', score=0.6, methods_stack=((DictionaryAnalyzer(), 'студентки', 40, 1),)),
 Parse(word='студентки', tag=OpencorporaTag('NOUN,anim,femn plur,nomn'), normal_form='студентка', score=0.4, methods_stack=((DictionaryAnalyzer(), 'студентки', 40, 7),))]

У класса MorphAnalyzer() есть метод parse, который возвращает что? Список экземпляров класса Parse. У этого класса есть свои атрибуты: word (исходная форма слова), tag (грам. инфа), normal_form (лемма), score(предпосчитанная на OpenCorpora вероятность правильности разбора) и несколько технических. 

Соответственно, получить информацию можно, просто обращаясь к атрибутам (не забудьте, что у нас всегда список, поэтому нужно еще и индекс разбора указывать):

In [6]:
print(parse[0].word)
print(parse[0].tag)
print(parse[0].normal_form)

студентки
NOUN,anim,femn sing,gent
студентка


Атрибут tag &ndash; это экземпляр класса OpencorporaTag, как можно догадаться. У него есть еще свои атрибуты, к которым тоже можно обращаться, чтобы получать более конкретную информацию о слове. 

In [10]:
parse = morph.parse('участник')

t = parse[0].tag  # я записала в переменную, просто чтобы не копировать каждый раз все целиком
# но это то же самое, что parse[0].tag.animacy...
print(f'Часть речи: {t.POS}')
print(f'Одушевленность: {t.animacy}\nПадеж: {t.case}\nРод: {t.gender}\nНаклонение: {t.mood}\
\nЧисло: {t.number}\nЛицо: {t.person}\nВремя: {t.tense}\nПереходность: {t.transitivity}\nЗалог: {t.voice}')

Часть речи: NOUN
Одушевленность: anim
Падеж: nomn
Род: masc
Наклонение: None
Число: sing
Лицо: None
Время: None
Переходность: None
Залог: None


In [8]:
parse = morph.parse('говорит')

t = parse[0].tag  
print(f'Часть речи: {t.POS}')
print(f'Одушевленность: {t.animacy}\nПадеж: {t.case}\nРод: {t.gender}\n\
Наклонение: {t.mood}\nЧисло: {t.number}\nЛицо: {t.person}\nВремя: {t.tense}\nПереходность: {t.transitivity}\nЗалог: {t.voice}')

Часть речи: VERB
Одушевленность: None
Падеж: None
Род: None
Наклонение: indc
Число: sing
Лицо: 3per
Время: pres
Переходность: tran
Залог: None


Если вы запрашиваете категорию, которой у данного слова нет (ну нет переходности у существительного), вернется None. 

Также можно попросить pymorphy поставить слово в конкретную форму или вообще вернуть всю парадигму. 

In [9]:
parse[0].inflect({'plur'})

Parse(word='говорят', tag=OpencorporaTag('VERB,impf,tran plur,3per,pres,indc'), normal_form='говорить', score=1.0, methods_stack=((DictionaryAnalyzer(), 'говорят', 415, 6),))

In [11]:
parse[0].lexeme  
# парадигму глагола лучше не выводить - она длинная; я перед запуском этой ячейки перезапустила разбор с существительным, поэтому не удивляйтесь. :)

[Parse(word='участник', tag=OpencorporaTag('NOUN,anim,masc sing,nomn'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участник', 2, 0),)),
 Parse(word='участника', tag=OpencorporaTag('NOUN,anim,masc sing,gent'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участника', 2, 1),)),
 Parse(word='участнику', tag=OpencorporaTag('NOUN,anim,masc sing,datv'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участнику', 2, 2),)),
 Parse(word='участника', tag=OpencorporaTag('NOUN,anim,masc sing,accs'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участника', 2, 3),)),
 Parse(word='участником', tag=OpencorporaTag('NOUN,anim,masc sing,ablt'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участником', 2, 4),)),
 Parse(word='участнике', tag=OpencorporaTag('NOUN,anim,masc sing,loct'), normal_form='участник', score=1.0, methods_stack=((DictionaryAnalyzer(), 'участник

Наконец, можно попросить pymorphy выводить грам. информацию по-русски:

In [12]:
parse[0].tag.cyr_repr

'СУЩ,од,мр ед,им'

Pymorphy очень быстро работает и имеет много возможностей, но совершенно не умеет разрешать омонимию и никак не учитывает контекст.

Алгоритм, легший в основу Mystem, разрабатывался в ИППИ и был первым вообще для русского языка; его в свое время купил у них Илья Сегалович, доработал, опубликовал собственную статью. Поисковик Яндекса когда-то работал на майстеме. Сам парсер написан в С (для скорости: бинарный поиск в питоне реализовать можно только с внешними библиотеками на С, а у майстема 2 словаря, по которым нужно искать). Для питона под него сделана оболочка (pymystem3). Майстем капризный, тяжело запускается, имеет не так много функций, но работает тоже довольно быстро и умеет доносить на бастардов: сообщать, что слово не найдено в его словаре. 

In [13]:
import pymystem3

m = pymystem3.Mystem(entire_input=False)

Майстем принимает только сырой текст в виде одной строки: у него встроенный токенизатор, потому что он пытается учитывать контекст. Поэтому стоит указывать entire_input=False при создании экземпляра класса, чтобы он не выводил вообще все, включая пробелы.

In [14]:
raw = '''Пердикка II (др.-греч. Περδίκκας Β΄ της Μακεδονίας) — македонский царь, правивший в 454—413 годах до н. э. После смерти Александра I среди его сыновей возник междоусобный конфликт, победителем из которого вышел Пердикка. На момент его воцарения Македония представляла собой отсталое государство, которому угрожала опасность завоевания как со стороны Афинского морского союза на юге, так и Одрисского царства на севере. На первых порах Пердикка был вынужден всеми силами избегать открытого вооружённого противостояния и лишь наблюдать за появлением множества греческих колоний на своих границах. С началом Пелопоннесской войны македонский царь с максимальной выгодой для государства использовал запутанные отношения между греческими полисами на Халкидиках, Афинами, Спартой и Коринфом. Пердикка не менее десяти раз заключал и расторгал союзы с основными участниками войны.'''

In [15]:
lemmas = m.lemmatize(raw)
analysis = m.analyze(raw)

In [16]:
lemmas[:10]  # надеюсь, греча вас тоже порадовала

['пердикк',
 'II',
 'др',
 'греча',
 'Περδίκκας',
 'Β',
 'της',
 'Μακεδονίας',
 'македонский',
 'царь']

In [17]:
analysis[:5]

[{'analysis': [{'lex': 'пердикк',
    'qual': 'bastard',
    'gr': 'S,имя,муж,од=(вин,ед|род,ед)'}],
  'text': 'Пердикка'},
 {'analysis': [], 'text': 'II'},
 {'analysis': [{'lex': 'др', 'gr': 'S,сокр,мн,неод=(пр|вин|дат|род|твор|им)'}],
  'text': 'др'},
 {'analysis': [{'lex': 'греча', 'gr': 'S,жен,неод=род,мн'}], 'text': 'греч'},
 {'analysis': [], 'text': 'Περδίκκας'}]

С леммами вроде все должно быть понятно, а что зашито в анализе?

Майстем возвращает список. Каждый токен в этом списке - это словарь с ключами analysis & text. Первого ключа может не быть: если у нас знак пунктуации. Если же он есть, то в нем содержится список (обычно состоящий из одного элемента - если не указать при создании экземпляра класса Mystem glue_grammar_info=False). 

In [18]:
print(f'Первое слово: {analysis[0]}')
print(f"Его грам. инфа: {analysis[0]['analysis']}\nЕго оригинальная форма: {analysis[0]['text']}")
print(f"Какие есть ключи в словаре с разбором: {analysis[0]['analysis'][0].keys()}")
print(f"Лемма: {analysis[0]['analysis'][0]['lex']}\n\
Этот ключ бывает только тогда, когда слова нет в словаре: {analysis[0]['analysis'][0]['qual']}\n\
А это грам. информация: {analysis[0]['analysis'][0]['gr']}")

Первое слово: {'analysis': [{'lex': 'пердикк', 'qual': 'bastard', 'gr': 'S,имя,муж,од=(вин,ед|род,ед)'}], 'text': 'Пердикка'}
Его грам. инфа: [{'lex': 'пердикк', 'qual': 'bastard', 'gr': 'S,имя,муж,од=(вин,ед|род,ед)'}]
Его оригинальная форма: Пердикка
Какие есть ключи в словаре с разбором: dict_keys(['lex', 'qual', 'gr'])
Лемма: пердикк
Этот ключ бывает только тогда, когда слова нет в словаре: bastard
А это грам. информация: S,имя,муж,од=(вин,ед|род,ед)


Непросто, да. Еще сложнее устроен ключ 'gr', который содержит грамматическую информацию о слове: обычно майстем склеивает варианты разбора, то есть, выше запись следует читать как "существительное, имя собственное, мужского рода, одушевленное; возможно, Acc Sg, а возможно, Gen Sg. 

Вот как раз если указать, чтобы грам. информация не склеивалась, майстем будет возвращать несколько словарей с вариантами по отдельности:

In [19]:
m_noglue = pymystem3.Mystem(entire_input=False, glue_grammar_info=False)

noglueanalysis = m_noglue.analyze(raw)
noglueanalysis[0]

{'analysis': [{'lex': 'пердикк',
   'qual': 'bastard',
   'gr': 'S,имя,муж,од=вин,ед'},
  {'lex': 'пердикк', 'qual': 'bastard', 'gr': 'S,имя,муж,од=род,ед'}],
 'text': 'Пердикка'}

Теперь о вещах, которых нет в стабильной версии Mystem, а есть только в той, которая устанавливается через git:

1. Майстем очень плохо умеет обрабатывать \n. Когда он получает строку, в которой много \n (а это неизбежно, ведь мы чаще хотим обрабатывать длиннющие тексты), на каждом \n он перезапускает свой бинарник (написанный в С). Поэтому на длинных текстах работать будет ОЧЕНЬ медленно (впрочем, все равно быстрее нейронок...). Чтобы решить эту проблему - ведь замена \n на пробелы, например, искажает контекст - сделали возможность особым образом обрабатывать \n, когда загружаем текст из файла. 
2. Есть функция, которая позволяет получить часть речи для конкретного токена. 

In [ ]:
analyze = m.analyze(file_path=path) # можно напрямую передавать в майстем путь к файлу с текстом - он сам откроет и обработает как ему надо

In [20]:
m.get_pos(analysis[0])

'S'

В 2017 году на соревновании конференции "Диалог" победила команда Гусев-Анастасьев: ребята создали морфопарсер для русского языка на нейронной сети. Он называется rnnmorph (RNN - это название модели нейронной сети, на которой он работает). Гусев при создании явно вдохновлялся pymorphy2 (кстати, Коробов для этого же соревнования сделал библиотеку для приведения разных тагсетов к одному): синтаксис rnnmorph очень похож на pymorphy2. 

In [21]:
from rnnmorph.predictor import RNNMorphPredictor

predictor = RNNMorphPredictor(language='ru')
parse = predictor.predict(['Пердикка', 'не', 'менее', 'десяти', 'раз', 'заключал', 'и', 'расторгал', 'союзы', 'с', 'основными', 'участниками', 'войны', '.'])
parse[:5]

2022-04-06 12:00:12.010260: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-06 12:00:12.010289: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-06 12:00:13.910063: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-06 12:00:13.910109: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-06 12:00:13.910134: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (aslin): /proc/driver/nvidia/version does not exist
2022-04-06 12:00:13.910346: I tensorflow/core/platform/cpu_feature_guar

[<normal_form=пердикк; word=Пердикка; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Sing; score=0.7922>,
 <normal_form=не; word=не; pos=PART; tag=_; score=1.0000>,
 <normal_form=менее; word=менее; pos=ADV; tag=Degree=Cmp; score=1.0000>,
 <normal_form=десять; word=десяти; pos=NUM; tag=Case=Gen; score=1.0000>,
 <normal_form=раз; word=раз; pos=NOUN; tag=Case=Gen|Gender=Masc|Number=Plur; score=0.9998>]

rnnmorph принимает список строк (токенов) и возвращает список объектов специального класса, у которого есть атрибуты normal_form (лемма), word (исходная форма), pos (часть речи), tag (грам. информация) и score (уверенность нейронной модели в правильности своего ответа). Он умеет снимать омонимию (лучше, чем майстем, но хуже, чем интегральный морфопарсер). 

In [22]:
parse[-2].pos  # например, можно узнать часть речи для предпоследнего слова в списке

'NOUN'

*Примечание*

RNNMorph, как и любая нейронная модель, умеет работать на GPU (видеокарте). 

Для любопытных: нейронные сети &ndash; это, по сути, бесконечное умножение гигантских матриц друг на друга. Нейронная сеть состоит из кучи нейронов-функций, и когда она должна выдать ответ, она получает входные данные в виде таких же матриц на первый слой, который состоит из миллиона нейронов, этот миллион нейрончиков кидается высчитывать результат зашитой внутри них функции (что-то похожее на ax + b), а их ответы получает второй слой таких же нейрончиков, и так пока не получится финальный ответ. То есть, мы производим миллиарды однотипных вычислений. Процессор видеокарты устроен как раз таким образом, что он супер-быстро умеет считать однотипные вещи (он считает их батчами &ndash; сразу пачками), то есть, он работает гораздо быстрее, чем CPU, но обучен именно однотипно считать. Поэтому нейронки обычно и работают на GPU, они просто созданы друг для друга!

Когда нейронка (и RNNMorph тоже) работает на видеокарте, она делает это обычно быстрее, чем на CPU. Но чтобы заставить RNNMorph перейти на видеокарту, нужно сложно настраивать ее (и иметь совместимую видеокарту до кучи); поэтому RNNMorph умеет, конечно, работать и на обычном процессоре, но при этом вываливает предупреждения про то, что настройки видеокарты он не нашел (они выше выделены красным). Их можно смело игнорировать! Это предупреждения, а не ошибки. 

Последнее, о чем мы поговорили - это формат разметки Universal Dependencies и UDPipe. Из питона запускать UDPipe поучимся в следующий раз, но у него есть версия онлайн на [сайте](https://lindat.mff.cuni.cz/services/udpipe/). 

[Universal Dependencies](https://universaldependencies.org/) - это грандиозный существующий с 2006 года проект (сперва чешских, а потом и самых разных лингвистов, у нас в России им активно занимается О. Ляшевская), который ставит целью разработать такой формат морфосинтаксической разметки, который был бы одинаково применим к самым разным языкам. То есть, основная его фича - это *единообразие*, из-за чего, к примеру, принято решение в русском языке частицу "не" считать advmod (так она себя ведет в германских языках...), а копулу не считать вершиной (потому что копула в агглютинативных языках обычно отсутствует, ср. русское "петя был учителем" vs турецкое "Petya öğretmendi", где -di - показатель прошедшего времени, присоединяющийся к *существительному*). 

UD для разметки использует формат файлов .conllu, которые представляют собой таблички (мы с таким на прошлых семинарах имели дело уже). В этом формате существует 10 колонок, каждая ячейка в строке отделяется знаком табуляции; предложения разделяются пустой строкой. На самом сайте UD очень много полезной информации, в том числе описание этого формата и сборник ссылок на приложения, которыми его удобно читать!

В питоне мы пытались написать некую читалку формата сами, но на самом деле, конечно, уже существуют готовые библиотеки для этого (и не одна, но мы посмотрим одну). 

In [ ]:
import pyconll

text = pyconll.load_from_file('myfile.conllu')

for sentence in text:
    for token in sentence:
        print(token.id, token.form, token.lemma, token.upos, token.feats, token.head, token.deprel)

Естественно, можно не только печатать информацию, но и добавлять в список и вообще делать все, что угодно. Что это за атрибуты у токенов?

- id - порядковый номер токена в предложении
- form - исходная форма
- lemma - лемма
- upos - часть речи в UD
- xpos - часть речи в неуниверсальном формате (обычно встречается, если датасет конвертированный)
- feats - грам. характеристики
- head - расстояние от синтаксической вершины
- deprel - тип синтаксической связи
- две зарезервированные ячейки

Где можно красивенько отрисовать .conllu файлы в виде деревьев зависимости:

[Арборатор](https://arborator.ilpga.fr/q.cgi): достаточно вставить текст в формате .conllu

[Conllu-Viewer на сайте UD](https://universaldependencies.org/conllu_viewer.html): умеет читать файлы и рисовать последовательно все предложения

Для затравки вот картинка с арборатора:

<img src='arbo.png'>